In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder\
    .appName("SparkSQL")\
    .config("spark.sql.debug.maxToStringFields", 2000)\
    .config("spark.driver.memory", "2g")\
    .getOrCreate()


In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.

from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
homeSales_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

# View complete descriptions in each cell
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 20)

# Show home_sales_revised data.
homeSales_df.show(truncate=False)


+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|id                                  |date      |date_built|price |bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+------------------------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d6-9c31-7398aa8f6089|2022-04-08|2016      |936923|4       |3        |3167       |11733   |2     |1         |76  |
|7530a2d8-1ae3-4517-9f4a-befe060c4353|2021-06-13|2013      |379628|2       |2        |2235       |14384   |1     |0         |23  |
|43de979c-0bf0-4c9f-85ef-96dc27b258d5|2019-04-12|2014      |417866|2       |2        |2127       |10575   |2     |0         |0   |
|b672c137-b88c-48bf-9f18-d0a4ac62fb8b|2019-10-16|2016      |239895|2       |2        |1631       |11149   |2     |0         |0   |
|e0726d4d-d595-4074-8283-4139a54d0d63|2022-01-08|2017      |424418|3       |2      

In [4]:
# Get datatypes
homeSales_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [5]:
# 2. Create a temporary view of the DataFrame.

homeSales_df.createOrReplaceTempView('homeSales')

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

query = """
SELECT year(date) as year, ROUND(AVG(price), 2) as avg_price
    FROM homeSales
    WHERE bedrooms = 4
    GROUP BY year(date)
    ORDER BY year
"""
spark.sql(query).show()


+----+---------+
|year|avg_price|
+----+---------+
|2019| 300263.7|
|2020|298353.78|
|2021|301819.44|
|2022|296363.88|
+----+---------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

query = """
SELECT date_built as built_year, ROUND(AVG(price), 2) as avg_price
    FROM homeSales
    WHERE bedrooms = 3 AND bathrooms = 3
    GROUP BY date_built
    ORDER BY date_built
"""
spark.sql(query).show()


+----------+---------+
|built_year|avg_price|
+----------+---------+
|      2010|292859.62|
|      2011|291117.47|
|      2012|293683.19|
|      2013|295962.27|
|      2014|290852.27|
|      2015| 288770.3|
|      2016|290555.07|
|      2017|292676.79|
+----------+---------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

query = """
SELECT date_built as built_year, ROUND(AVG(price), 2) as avg_price
    FROM homeSales
    WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
    ORDER BY date_built
"""
spark.sql(query).show()

+----------+---------+
|built_year|avg_price|
+----------+---------+
|      2010|285010.22|
|      2011|276553.81|
|      2012|307539.97|
|      2013|303676.79|
|      2014|298264.72|
|      2015|297609.97|
|      2016| 293965.1|
|      2017|280317.58|
+----------+---------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
#    where the homes are greater than or equal to $350,000?
#    Although this is a small dataset, determine the run time for this query.

from pyspark.sql.functions import col

start_time = time.time()

query = """
SELECT CAST(view AS INT) AS view, ROUND(AVG(price), 2) AS avg_price
    FROM homeSales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))
un_Cached_runtime = (time.time() - start_time)

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.2711727619171143 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.

spark.sql("cache table homeSales")

""


In [11]:
# 8. Check if the table is cached.
# spark.catalog.isCached('home_sales')

spark.catalog.isCached('homeSales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000.
# Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """
SELECT CAST(view AS INT) AS view, ROUND(AVG(price), 2) AS avg_price
    FROM homeSales
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))
Cached_runtime = (time.time() - start_time)

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 0.7868280410766602 seconds ---


In [13]:
# Runtime comparisons: UnCached versus Cached

print("Uncached Runtime: --- %s seconds ---" % un_Cached_runtime, "\n\n  Cached Runtime: --- %s seconds ---" % Cached_runtime)


Uncached Runtime: --- 1.2713348865509033 seconds --- 

  Cached Runtime: --- 0.7870340347290039 seconds ---


In [14]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

homeSales_df.write.partitionBy("date_built").mode("overwrite").parquet("parquet_homeSales")

In [15]:
# 11. Read the parquet formatted data.

pt_homeSales_pq_df = spark.read.parquet('parquet_homeSales')

In [16]:
# 12. Create a temporary table for the parquet data.
# Convert the DataFrame to a view.
pt_homeSales_pq_df.createOrReplaceTempView('p_homeSales_p')

In [17]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query = """
SELECT CAST(view AS INT) AS view, ROUND(AVG(price), 2) AS avg_price
    FROM p_homeSales_p
    WHERE price >= 350000
    GROUP BY view
    ORDER BY view DESC
"""
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

part_parq_runtime = (time.time() - start_time)

+----+----------+
|view| avg_price|
+----+----------+
| 100| 1026669.5|
|  99|1061201.42|
|  98|1053739.33|
|  97|1129040.15|
|  96|1017815.92|
|  95| 1054325.6|
|  94| 1033536.2|
|  93|1026006.06|
|  92| 970402.55|
|  91|1137372.73|
|  90|1062654.16|
|  89|1107839.15|
|  88|1031719.35|
|  87| 1072285.2|
|  86|1070444.25|
|  85|1056336.74|
|  84|1117233.13|
|  83|1033965.93|
|  82| 1063498.0|
|  81|1053472.79|
+----+----------+
only showing top 20 rows

--- 1.3078515529632568 seconds ---


In [18]:
# Runtime comparisons: partition_parquet_Cached versus Cached

print("                  Cached Runtime: --- %s seconds ---" % Cached_runtime, "\n\npartition_parquet_Cached Runtime: --- %s seconds ---" % part_parq_runtime)

                  Cached Runtime: --- 0.7870340347290039 seconds --- 

partition_parquet_Cached Runtime: --- 1.3081071376800537 seconds ---


### Un-Cache

In [19]:
# 14. Uncache the home_sales temporary table.

spark.sql("uncache table homeSales")

""


In [20]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("homeSales")

False

## Analysis of Runtime

<pre><span style="font-size: 12px;">
*                     Uncached Runtime:  1.2713 seconds<br>
*                       Cached Runtime:  0.7870 seconds<br>
*     partition_parquet_Cached Runtime:  1.3081 seconds<br>
</pre>
</span>


<span style="font-size: 14px;">
Uncached Runtime of a query doesn't have the advantage of the data stored in memory, hence a longer runtime compared to Cached query.</span><br>
<br>
<span style="font-size: 14px;">
Caching data means storing a copy of data in RAM for fast access.<br>
Therefore, the run time is less than uncached query.</span><br>
<br>
<span style="font-size: 14px;">
Partititioning data is expected to organize the data so as to target only relevent data for specific queries.  In our case, the column "date_built", which was not useful in the query, is partitioned.<br>
So the runtime became longer than for the un-partitioned, although it was a cached table.  <br>In this case, the performance was worse than, when the table was NOT cached.<br>
</span>